# Relatório 2 - Algoritmo Naive Bayes
### Aluno: José Ivo Schwade Araújo

Importando as bibliotecas necessárias:

In [1]:
# Importação das bibliotecas necessárias
import numpy as np                                            # Biblioteca para operações numéricas
import pandas as pd                                           # Biblioteca para manipulação de dados em DataFrames
from sklearn.model_selection import train_test_split          # Função para dividir os dados em treino e teste
from sklearn.feature_extraction.text import CountVectorizer   # Transforma texto em representações vetoriais
from sklearn.naive_bayes import BernoulliNB                   # Classificador Naive Bayes com distribuição de Bernoulli
from sklearn.neighbors import KNeighborsClassifier            # Classificador k-NN
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report  # Métricas de avaliação
import matplotlib.pyplot as plt                               # Biblioteca de visualização gráfica
import seaborn as sns                                         # Biblioteca de visualização baseada em matplotlib, com estilo melhorado
# Para figuras geradas pelo matplotlib serem exibidas diretamente no notebook
%matplotlib inline

In [2]:
# Carregamento do dataset de spam
url = "https://raw.githubusercontent.com/justmarkham/pycon-2016-tutorial/master/data/sms.tsv"
df = pd.read_csv(url, sep='\t', header=None, names=['label', 'message'])


Preparação dos dados:

In [4]:
# Exemplo do CountVectorizer

# Frases de treino
X_train = [
    "maçã e banana",
    "banana é boa",
    "eu gosto de maçã",
    "banana e maçã são frutas"
]

# Frases de teste
X_test = [
    "eu gosto de banana",
    "laranja é uma fruta",
    "maçã verde é azeda"
]

# Inicialização do vetor com binarização
vectorizer = CountVectorizer(binary=True)  # binary=True: CountVectorizer marca apenas se uma palavra apareceu (1) ou não apareceu (0) em um determinado documento, sem contar quantas vezes ela apareceu.

# Ajuste + transformação do treino
X_train_vec = vectorizer.fit_transform(X_train)  # matriz esparsa

# Somente transformação do teste com o mesmo vocabulário
X_test_vec = vectorizer.transform(X_test)       # matriz esparsa

# Criação dos DataFrames para visualização
df_train = pd.DataFrame(
    X_train_vec.toarray(),  # Transforma em matriz densa
    columns=vectorizer.get_feature_names_out(),
    index=[f"treino_{i+1}" for i in range(len(X_train))]
)

df_test = pd.DataFrame(
    X_test_vec.toarray(),   # Transforma em matriz densa
    columns=vectorizer.get_feature_names_out(),
    index=[f"teste_{i+1}" for i in range(len(X_test))]
)

# Visualizações
print("Vocabulário aprendido:")
print(vectorizer.get_feature_names_out())

print("\nMatriz binária - Dados de treino:")
display(df_train)

print("\nMatriz binária - Dados de teste:")
display(df_test)  # Usa o mesmo vocabulário aprendido no fit do treino, e ignora palavras desconhecidas

# Conversão das labels para valores binários
df['label'] = df['label'].map({'ham': 0, 'spam': 1})

# Divisão dos dados em treino e teste
X_train, X_test, y_train, y_test = train_test_split(  # Divide dados em treino (70%) e teste (30%)
    df['message'],  # Variável independente: textos das mensagens
    df['label'],    # Variável dependente: rótulo binário
    test_size=0.3,  # Proporção de teste: 30%
    random_state=42 # Semente para reprodutibilidade dos resultados
)

# Transformação dos textos em vetores de contagem de palavras (binário)
vectorizer = CountVectorizer(binary=True)         # Inicializa o vetor com contagem binária (presença/ausência da palavra)
X_train_vec = vectorizer.fit_transform(X_train)   # Ajusta e transforma os dados de treino em vetores binários
X_test_vec = vectorizer.transform(X_test)         # Transforma os dados de teste com o mesmo vocabulário aprendido no treino

feature_names = vectorizer.get_feature_names_out()
X_train_df = pd.DataFrame(X_train_vec.toarray(),
                          columns=feature_names)
print(f'"Vocabulário aprendido:{feature_names}\n')
display(X_train_df.tail(113))



Vocabulário aprendido:
['banana' 'boa' 'de' 'eu' 'frutas' 'gosto' 'maçã' 'são']

Matriz binária - Dados de treino:


,banana,boa,de,eu,frutas,gosto,maçã,são
treino_1,1,0,0,0,0,0,1,0
treino_2,1,1,0,0,0,0,0,0
treino_3,0,0,1,1,0,1,1,0
treino_4,1,0,0,0,1,0,1,1



Matriz binária - Dados de teste:


,banana,boa,de,eu,frutas,gosto,maçã,são
teste_1,1,0,1,1,0,1,0,0
teste_2,0,0,0,0,0,0,0,0
teste_3,0,0,0,0,0,0,1,0


"Vocabulário aprendido:['00' '000' '000pes' ... 'zouk' 'zyada' 'èn']



,00,000,000pes,008704050406,0089,0121,01223585236,01223585334,02,0207,...,zed,zeros,zhong,zindgi,zoe,zogtorius,zoom,zouk,zyada,èn
3787,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3788,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3789,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3790,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3791,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3895,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3896,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3897,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3898,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


Testando as outras métricas para o algoritmo KNN:

In [ ]:
# Treinamento dos modelos
models = {
    "K-Nearest Neighbors": KNeighborsClassifier(),
}

# Treinamento e avaliação de cada modelo
results = {"Bernoulli Naive Bayes": {"accuracy": accuracy_nb, "classification_report": classification_report(y_test, y_pred_nb), "confusion_matrix": conf_matrix_nb}}
for model_name, model in models.items():
    model.fit(X_train_vec, y_train)
    y_pred = model.predict(X_test_vec)
    accuracy = accuracy_score(y_test, y_pred)
    results[model_name] = {
        "accuracy": accuracy,
        "classification_report": classification_report(y_test, y_pred),
        "confusion_matrix": confusion_matrix(y_test, y_pred)
    }
    print(f"Modelo: {model_name}")
    print(f"Precisão: {accuracy:.4f}")
    print("Relatório de Classificação:")
    print(results[model_name]["classification_report"])
    print("Matriz de Confusão:")
    print(results[model_name]["confusion_matrix"])
    print("-" * 60)


NameError: name 'results' is not defined